In [ ]:
import requests
import json
import numpy as np

def getAddress(x, y):
  # 1. API 기본 정보 설정
  URL = "https://dapi.kakao.com/v2/local/geo/coord2regioncode.json"
  API_KEY = "xxxx" # 실제 API 키로 대체 필요

  # 2. 헤더 설정 (인증 키 포함)
  headers = {
    "Authorization": f"KakaoAK {API_KEY}"
  }

  # 3. 쿼리 파라미터 설정
  # 'x'와 'y'는 URL에 직접 포함되거나 params로 전달 가능합니다.
  params = {
    "x": x,
    "y": y
  }

  # 4. GET 요청 보내기
  try:
    response = requests.get(URL, headers=headers, params=params)
    
    # HTTP 상태 코드 확인
    response.raise_for_status()
    
    # 5. JSON 응답 출력
    data = response.json()
    # 한글 깨짐 없이, 보기 좋게 들여쓰기하여 출력
    json_obj = json.dumps(data, indent=2, ensure_ascii=False)
    #print(json_obj)
    #print(data['documents'][0]['region_1depth_name'])
    #print(data['documents'][0]['region_2depth_name'])
    #print(data['documents'][0]['region_3depth_name'])
    
    return data['documents'][0]['region_1depth_name'], data['documents'][0]['region_2depth_name'], data['documents'][0]['region_3depth_name']

  except requests.exceptions.RequestException as e:
    print(f"API 요청 오류 발생: {e}")
    if 'response' in locals():
      print(f"응답 상태 코드: {response.status_code}")
      print(f"응답 본문: {response.text}")
      
    return np.nan, np.nan, np.nan

In [30]:
print(getAddress(126.9079339, 37.64672726))

('서울특별시', '은평구', '진관동')


In [13]:
import pandas as pd
import numpy as np

src_df = pd.read_csv('서울시버스노선별정류소정보(20251002) - Data.csv')

In [14]:
src_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41261 entries, 0 to 41260
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   ROUTE_ID  41261 non-null  int64  
 1   노선명       41261 non-null  object 
 2   순번        41261 non-null  int64  
 3   NODE_ID   41261 non-null  int64  
 4   ARS_ID    41261 non-null  int64  
 5   정류소명      41261 non-null  object 
 6   X좌표       41261 non-null  float64
 7   Y좌표       41261 non-null  float64
dtypes: float64(2), int64(4), object(2)
memory usage: 2.5+ MB


In [15]:
src_df['시'] = np.nan
src_df['구'] = np.nan
src_df['동'] = np.nan


In [16]:
src_df.head()

,ROUTE_ID,노선명,순번,NODE_ID,ARS_ID,정류소명,X좌표,Y좌표,시,구,동
0,100100407,240,1,106000319,7418,중랑공영차고지.신내역,127.1028,37.613545,NaN,NaN,NaN
1,100100049,273,1,106000319,7418,중랑공영차고지.신내역,127.1028,37.613545,NaN,NaN,NaN
2,100100186,2012,1,106000319,7418,중랑공영차고지.신내역,127.1028,37.613545,NaN,NaN,NaN
3,100100190,2015,1,106000319,7418,중랑공영차고지.신내역,127.1028,37.613545,NaN,NaN,NaN
4,100100599,2311,1,106000319,7418,중랑공영차고지.신내역,127.1028,37.613545,NaN,NaN,NaN


In [31]:
import pandas as pd
import requests
import time
import json
from io import StringIO
from math import isnan # NaN 값 비교를 위해 임포트

In [32]:
# 3. API 결과를 저장할 딕셔너리 초기화
# 키: (x, y) 튜플, 값: {'a': a_val, 'b': b_val, 'c': c_val}
api_cache = {}

# 4. API 호출 대상 좌표 목록 추출
# 'a' 컬럼 값이 NaN인 행을 필터링하고, 'x', 'y' 컬럼의 중복을 제거합니다.
# isnan을 사용해 NaN인지 확인
coords_to_query = (
    src_df[src_df['시'].apply(lambda x: isnan(x) if pd.notna(x) else True)] # 'a'가 NaN인 행 필터링
    [['X좌표', 'Y좌표']]
    .drop_duplicates()
    .itertuples(index=False, name=None)
)

coords_list = list(coords_to_query)
print(f"API 호출이 필요한 고유 좌표 쌍: {len(coords_list)}개")

API 호출이 필요한 고유 좌표 쌍: 12894개


In [33]:
# 5. 고유 좌표에 대해서만 API 호출
# 다음 실행할때는 tqdm 쓰고 로그는 제거하자
for x_val, y_val in coords_list:
    coord_key = (x_val, y_val)
    #print(f"\n📡 API 호출 시작: x={x_val}, y={y_val}")
    
    try:
        # NOTE: 응답 구조에 맞게 수정 필요
        a_val, b_val, c_val = getAddress(x_val, y_val)
        
        # 4. 캐시에 결과 저장
        api_cache[coord_key] = {'시': a_val, '구': b_val, '동': c_val}
        #print(f"✅ 처리 완료. 결과: 시={a_val}, 구={b_val}, 동={c_val}")

    except Exception as e:
        print(f"❌ 처리 중 오류 발생 ({coord_key}): {e}")
        api_cache[coord_key] = {'a': None, 'b': None, 'c': None, 'api_status': f"Error: {str(e)}"}
        
    # API 호출 제한 방지를 위한 지연 시간 (필수 권장!)
    time.sleep(0.5)


📡 API 호출 시작: x=127.1027997, y=37.61354475
✅ 처리 완료. 결과: 시=서울특별시, 구=중랑구, 동=신내동

📡 API 호출 시작: x=127.0407624, y=37.65893642
✅ 처리 완료. 결과: 시=서울특별시, 구=도봉구, 동=쌍문동

📡 API 호출 시작: x=126.969153, y=37.556235
✅ 처리 완료. 결과: 시=서울특별시, 구=중구, 동=만리동1가

📡 API 호출 시작: x=126.93783, y=37.46571
✅ 처리 완료. 결과: 시=서울특별시, 구=관악구, 동=신림동

📡 API 호출 시작: x=126.9398176, y=37.47037897
✅ 처리 완료. 결과: 시=서울특별시, 구=관악구, 동=신림동

📡 API 호출 시작: x=126.919247, y=37.489233
✅ 처리 완료. 결과: 시=서울특별시, 구=관악구, 동=신림동

📡 API 호출 시작: x=127.0503112, y=37.62754013
✅ 처리 완료. 결과: 시=서울특별시, 구=노원구, 동=월계동

📡 API 호출 시작: x=126.956215, y=37.491679
✅ 처리 완료. 결과: 시=서울특별시, 구=동작구, 동=상도동

📡 API 호출 시작: x=126.892827, y=37.490156
✅ 처리 완료. 결과: 시=서울특별시, 구=구로구, 동=구로동

📡 API 호출 시작: x=126.9572808, y=37.49531836
✅ 처리 완료. 결과: 시=서울특별시, 구=동작구, 동=상도동

📡 API 호출 시작: x=126.92119, y=37.453078
✅ 처리 완료. 결과: 시=서울특별시, 구=금천구, 동=시흥동

📡 API 호출 시작: x=127.1762683, y=37.3455333
✅ 처리 완료. 결과: 시=경기도, 구=광주시, 동=능평동

📡 API 호출 시작: x=126.8688211, y=37.49056774
✅ 처리 완료. 결과: 시=경기도, 구=광명시, 동=철산동

📡 API 호출 시

In [34]:
api_cache

{(127.1027997, 37.61354475): {'시': '서울특별시', '구': '중랑구', '동': '신내동'},
 (127.0407624, 37.65893642): {'시': '서울특별시', '구': '도봉구', '동': '쌍문동'},
 (126.969153, 37.556235): {'시': '서울특별시', '구': '중구', '동': '만리동1가'},
 (126.93783, 37.46571): {'시': '서울특별시', '구': '관악구', '동': '신림동'},
 (126.9398176, 37.47037897): {'시': '서울특별시', '구': '관악구', '동': '신림동'},
 (126.919247, 37.489233): {'시': '서울특별시', '구': '관악구', '동': '신림동'},
 (127.0503112, 37.62754013): {'시': '서울특별시', '구': '노원구', '동': '월계동'},
 (126.956215, 37.491679): {'시': '서울특별시', '구': '동작구', '동': '상도동'},
 (126.892827, 37.490156): {'시': '서울특별시', '구': '구로구', '동': '구로동'},
 (126.9572808, 37.49531836): {'시': '서울특별시', '구': '동작구', '동': '상도동'},
 (126.92119, 37.453078): {'시': '서울특별시', '구': '금천구', '동': '시흥동'},
 (127.1762683, 37.3455333): {'시': '경기도', '구': '광주시', '동': '능평동'},
 (126.8688211, 37.49056774): {'시': '경기도', '구': '광명시', '동': '철산동'},
 (126.8688129, 37.49065738): {'시': '경기도', '구': '광명시', '동': '철산동'},
 (126.989289, 37.55974): {'시': '서울특별시', '구': '중구', '동': '예장동'

In [35]:
def apply_api_result(row):
    coord_key = (row['X좌표'], row['Y좌표'])
    
    # 'a' 컬럼값이 이미 존재하는지 확인 (NaN이 아닌 경우)
    if pd.notna(row['시']):
        return row
    
    # 캐시에서 결과를 찾아서 적용
    if coord_key in api_cache:
        result = api_cache[coord_key]
        row['시'] = result.get('시', row['시'])
        row['구'] = result.get('구', row['구'])
        row['동'] = result.get('동', row['동'])
    
    return row

In [36]:
# 6. 데이터프레임에 결과 일괄 적용
print("\n--- 결과 일괄 적용 중 ---")

# apply 함수를 사용하여 모든 행에 결과를 적용합니다.
src_df = src_df.apply(apply_api_result, axis=1)

# 7. 최종 결과 출력
print("\n=== 최종 결과 데이터프레임 ===")
print(src_df.head())


--- 결과 일괄 적용 중 ---

=== 최종 결과 데이터프레임 ===
    ROUTE_ID   노선명  순번    NODE_ID  ARS_ID         정류소명       X좌표        Y좌표  \
0  100100407   240   1  106000319    7418  중랑공영차고지.신내역  127.1028  37.613545   
1  100100049   273   1  106000319    7418  중랑공영차고지.신내역  127.1028  37.613545   
2  100100186  2012   1  106000319    7418  중랑공영차고지.신내역  127.1028  37.613545   
3  100100190  2015   1  106000319    7418  중랑공영차고지.신내역  127.1028  37.613545   
4  100100599  2311   1  106000319    7418  중랑공영차고지.신내역  127.1028  37.613545   

       시    구    동  
0  서울특별시  중랑구  신내동  
1  서울특별시  중랑구  신내동  
2  서울특별시  중랑구  신내동  
3  서울특별시  중랑구  신내동  
4  서울특별시  중랑구  신내동  


In [37]:
src_df

,ROUTE_ID,노선명,순번,NODE_ID,ARS_ID,정류소명,X좌표,Y좌표,시,구,동
0,100100407,240,1,106000319,7418,중랑공영차고지.신내역,127.102800,37.613545,서울특별시,중랑구,신내동
1,100100049,273,1,106000319,7418,중랑공영차고지.신내역,127.102800,37.613545,서울특별시,중랑구,신내동
2,100100186,2012,1,106000319,7418,중랑공영차고지.신내역,127.102800,37.613545,서울특별시,중랑구,신내동
3,100100190,2015,1,106000319,7418,중랑공영차고지.신내역,127.102800,37.613545,서울특별시,중랑구,신내동
4,100100599,2311,1,106000319,7418,중랑공영차고지.신내역,127.102800,37.613545,서울특별시,중랑구,신내동
...,...,...,...,...,...,...,...,...,...,...,...
41256,100100589,N61,183,116000142,17233,개봉역,126.856720,37.493978,서울특별시,구로구,개봉동
41257,100100589,N61,184,116000144,17235,개봉1동사거리.개봉푸르지오아파트,126.846791,37.501413,서울특별시,구로구,개봉동
41258,100100589,N61,185,114000125,15228,신정학마을아파트,126.843963,37.507421,서울특별시,양천구,신정동
41259,100100589,N61,186,114000257,15362,신정현대아파트.서부트럭터미널,126.842007,37.509011,서울특별시,양천구,신정동


In [ ]:
src_df.to_csv('20251014_서울시_버스정류장_주소.csv', index=False, encoding='utf-8')